# below are the variant calling tutorials of  exome samples on hg19

this tutorial is under /home/hq2130/tutorials

### Overall, we need a workflow driver to automate the process. 

# step 1: from fastq to bams

need input file with read group info, need bwa as executable, output is a bam in bamname.align folder

In [1]:
# fastq files are under fastq folder
!ls -lh fastq/

total 22G
-rwxrwx--- 1 hq2130 shenlab 1.6G Jul 12  2018 1-04687-01_P1.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 1.8G Jul 12  2018 1-04687-01_P2.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 2.0G Jul 12  2018 1-04687-02_P1.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 2.3G Jul 12  2018 1-04687-02_P2.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 1.5G Jul 12  2018 1-04687_P1.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 1.8G Jul 12  2018 1-04687_P2.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 1.7G Jul 12 15:21 1-07134-01_P1.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 2.0G Jul 12 15:22 1-07134-01_P2.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 1.5G Jul 12 15:22 1-07134-02_P1.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 1.8G Jul 12 15:23 1-07134-02_P2.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 1.7G Jul 12 15:23 1-07134_P1.fastq.gz
-rwxrwx--- 1 hq2130 shenlab 2.0G Jul 12 15:24 1-07134_P2.fastq.gz


In [2]:
### NEED to add how to generate this file 
# read1_location, read_group, read2_location
!cat fastq_head.txt

/home/hq2130/tutorials/fastq/1-07134-02_P1.fastq.gz	@RG\tID:1-07134-02\tPL:ILLUMINA\tPU:K00175:13\tLB:ATGTAA+TGACAT\tSM:1-07134-02\tCN:COLUMBIA_YS_LAB	/home/hq2130/tutorials/fastq/1-07134-02_P2.fastq.gz
/home/hq2130/tutorials/fastq/1-07134_P1.fastq.gz	@RG\tID:1-07134\tPL:ILLUMINA\tPU:K00175:13\tLB:ATGTCA+TGACAT\tSM:1-07134\tCN:COLUMBIA_YS_LAB	/home/hq2130/tutorials/fastq/1-07134_P2.fastq.gz
/home/hq2130/tutorials/fastq/1-07134-01_P1.fastq.gz	@RG\tID:1-07134-01\tPL:ILLUMINA\tPU:K00175:13\tLB:AGTTGC+GGAACT\tSM:1-07134-01\tCN:COLUMBIA_YS_LAB	/home/hq2130/tutorials/fastq/1-07134-01_P2.fastq.gz
/home/hq2130/tutorials/fastq/1-04687_P1.fastq.gz	@RG\tID:1-04687\tPL:ILLUMINA\tPU:K00175:9\tLB:CGATGT+ACATCG\tSM:1-04687\tCN:COLUMBIA_YS_LAB	/home/hq2130/tutorials/fastq/1-04687_P2.fastq.gz
/home/hq2130/tutorials/fastq/1-04687-02_P1.fastq.gz	@RG\tID:1-04687-02\tPL:ILLUMINA\tPU:K00162:21\tLB:GCCAAT+NTTGGC\tSM:1-04687-02\tCN:COLUMBIA_YS_LAB	/home/hq2130/tutorials/fastq/1-04687-02_P2.fastq.gz
/home/hq21

In [4]:
## submit array jobs to cluster
### pay attention to number of CPU threads. Default is 1. 
EXOMPPLN='/home/yufengshen/CUMC/Exome-pipeline-Jiayao'
!qsub  -t 1-6 $EXOMPPLN/ExmAln.1a.Align_Fastq_to_Bam_with_BWAmem.sh -i fastq_head.txt -r $EXOMPPLN/WES_Pipeline_References.b37.biocluster.sh

Your job-array 859297.1-6:1 ("BWAMEM") has been submitted


# step 2: bams to gvcf
pay attentions to the target file, needs tabix, bgzip

In [5]:
## get the list of bam files. 
!cat bam.txt

/home/hq2130/tutorials/bam/1-04687-01.bam
/home/hq2130/tutorials/bam/1-04687-02.bam
/home/hq2130/tutorials/bam/1-04687.bam
/home/hq2130/tutorials/bam/1-07134-01.bam
/home/hq2130/tutorials/bam/1-07134-02.bam
/home/hq2130/tutorials/bam/1-07134.bam


In [8]:
### GATK haplotypecaller to generate gVCF file for each sample.
### We should investigate how much RAM is really needed for exome or WGS samples. 

EXOMPPLN='/home/yufengshen/CUMC/Exome-pipeline-Jiayao'
!qsub  -t 1-6 $EXOMPPLN/ExmAln.2.HaplotypeCaller_GVCFmode.sh -i bam.txt -r $EXOMPPLN/WES_Pipeline_References.b37.biocluster.sh -t src/MedExome_hg19_capture_targets_reformat.bed

Your job-array 859300.1-6:1 ("HaplotypeCaller_GVCFmode") has been submitted


# step 3: Merge individual gvcf to large gvcfs
This step is to reduce memory usage in variant calling, GATK recommand combine 200 gvcfs for exome, for whole genome, combine a trio
the inputs need to end with 'list'

In [25]:
### list of lists. Each list contains the files to be merged. 
!cat gvcf.list

test_00.list
test_01.list


In [26]:
!cat test_00.list

/home/hq2130/tutorials/gvcf/1-04687-01.g.vcf.gz
/home/hq2130/tutorials/gvcf/1-04687-02.g.vcf.gz
/home/hq2130/tutorials/gvcf/1-04687.g.vcf.gz


In [27]:
EXOMPPLN='/home/yufengshen/CUMC/Exome-pipeline-Jiayao'
!qsub  -t 1-2  $EXOMPPLN/ExmAdHoc.6.Merge_gVCFs.sh -i gvcf.list -r $EXOMPPLN/WES_Pipeline_References.b37.biocluster.sh

Your job-array 859312.1-2:1 ("MergeGVCF") has been submitted


# step 4: gvcfs -> vcf, each vcf corresponding to the region  

In [28]:
## list of merged gVCF files
!cat cmbgvcf.list

/home/hq2130/tutorials/cmbgvcf/test_00.Combined.g.vcf.gz
/home/hq2130/tutorials/cmbgvcf/test_01.Combined.g.vcf.gz


In [30]:
### GATK Genotyper.  We are considering to replace it with GLNexus. 
### We need a mechanism to automatically detect failed jobs and re-submit. 
### We should investigate how much RAM is really needed for exome or WGS samples. 
# split the target file into 10 chunks, pay attention to option -j and -t
EXOMPPLN='/home/yufengshen/CUMC/Exome-pipeline-Jiayao'
!qsub -t 1-10  $EXOMPPLN/ExmVC.1hc.GenotypeGVCFs.sh -j 10 -i cmbgvcf.list -r $EXOMPPLN/WES_Pipeline_References.b37.biocluster.sh -t src/MedExome_hg19_capture_targets_reformat.bed

Your job-array 859314.1-10:1 ("JointGenotyping") has been submitted


# step 5: combine split vcfs

In [33]:
EXOMPPLN='/home/yufengshen/CUMC/Exome-pipeline-Jiayao'
!qsub $EXOMPPLN/ExmVC.2.MergeVCF.sh -i cmbgvcf.splitfiles/ -r $EXOMPPLN/WES_Pipeline_References.b37.biocluster.sh

Your job 859317 ("MergeVCF") has been submitted


# step 6 VQSR(optional)

not very useful

In [34]:
!qsub $EXOMPPLN/ExmVC.4.RecalibrateVariantQuality.sh -i cmbgvcf.rawvariants.vcf.gz  -r $EXOMPPLN/WES_Pipeline_References.b37.biocluster.sh

Your job 859318 ("VQSR") has been submitted


# step 7: Annotate vcf with ANNOVAR
ask jiayao for how to annotate with annovar on large files, need to split first then annotate then combine

In [36]:
### default, each line is a position. If there are multiple alternative alleles, the corresponding annotations are separately by , 
!qsub  $EXOMPPLN/Test.AnnotateVCF_direct.sh -i cmbgvcf.rawvariants.recalibrated.vcf -r $EXOMPPLN/WES_Pipeline_References.b37.biocluster.sh

Your job 859320 ("ANNOVAR") has been submitted
